In [ ]:
import os  # operating system functions like renaming files and directories
import shutil  # recursive file and directory operations
import glob  # pattern matching for paths
import pandas as pd  # data mangling and transforming
import bandicoot as bc  # MIT toolkit for creating bandicoot indicators
import argparse  # entering flags from the cmd line
from pyspark.sql import SparkSession  # using spark context for big data files
from pyspark.sql.functions import col  # needed for function over each column
from attributes_classes import Attributes
from queries import Queries
from preprocessing_functions import (read_as_sdf, sdf_from_folder, union_all,
                                     bc_batch, aggregate_chunks)

In [ ]:
mp_flag = True
bc_flag = True
no_info = False
clean_up = False
raw_data_path = '../data/'

In [ ]:
# define attributes for this session
attributes = Attributes(mp_flag=mp_flag, bc_flag=bc_flag, no_info=no_info, clean_up=clean_up,
                        raw_data_path=raw_data_path,
                        cap_lat=15.500654, cap_long=32.559899,  # capital gps
                        weekend_days=[5, 6], call_unit_multiplicator=60)

In [ ]:
q = Queries()